# Create full 16S database by merging SILVA, RDP and Greengenes

In [2]:
from utils import check_dir
import utils
from os.path import join, exists, basename, splitext
import pickle
from re import sub, search
from copy import copy
%load_ext autoreload

## Original Databases

### Download original files
We download greengenes and silva 16S database (99NR).

- Greengenes v13_8 (99)
- Silva v138 SSURef NR99 515F/806R (qiime2  pre-formatted)

RDP database was obtained from the `RDPClassifier_16S_trainsetNo18_QiimeFormat.zip` file in this [website](https://sourceforge.net/projects/rdp-classifier/files/RDP_Classifier_TrainingData)

In [2]:
# Environment
or_dir = "original_db"
check_dir(or_dir)

download_db = {"greengenes":[
                ("http://greengenes.microbio.me/greengenes_release/gg_13_8_otus/rep_set/99_otus.fasta",
                 join(or_dir, "gg_13_8_NR99_seqs.fasta")),
                ("http://greengenes.microbio.me/greengenes_release/gg_13_8_otus/taxonomy/99_otu_taxonomy.txt",
                join(or_dir, "gg_13_8_NR99_taxa.txt"))
],
             "silva":[
                ("https://data.qiime2.org/2022.8/common/silva-138-99-seqs.qza",
                join(or_dir, "silva_138_NR99_seqs.qza")),
                ("https://data.qiime2.org/2022.8/common/silva-138-99-tax.qza",
                join(or_dir, "silva_138_NR99_taxa.qza"))
            ]
}

In [3]:
for db, values in download_db.items():
    for item in values:
        link = item[0]
        path = item[1]
        
        ! wget -nc {link} -O {outname}

--2023-04-06 20:24:52--  http://%7Blink%7D/
Resolving {link} ({link})... failed: Name or service not known.
wget: unable to resolve host address ‘{link}’
File ‘{outname}’ already there; not retrieving.
File ‘{outname}’ already there; not retrieving.
File ‘{outname}’ already there; not retrieving.


### Decompress silva qza files

In [ ]:
# Outnames
silva_fasta_qza = download_db['silva'][0][1]
silva_taxa_qza = download_db['silva'][1][1]
silva_fasta = f"{splitext(download_db['silva'][0][1])[0]}.fasta"
silva_txt = f"{splitext(download_db['silva'][1][1])[0]}.txt"

utils.unqza(silva_fasta_qza, silva_fasta)
utils.unqza(silva_taxa_qza, silva_taxa)

### Original databases files

In [3]:
or_dir = "original_db"
databases = {
    'silva': [join(or_dir, 'silva_138_NR99_seqs.fasta'),
             join(or_dir, 'silva_138_NR99_taxa.txt')
             ],
    'gg': [join(or_dir, "gg_13_8_NR99_seqs.fasta"),
           join(or_dir, "gg_13_8_NR99_taxa.txt")
          ],
    'gp_97': [join(or_dir, "gp_97_otus_OLD.fasta"),
           join(or_dir, "gp_97_taxa_OLD.txt")       
             ],
    'rdp': [join(or_dir, 'RDPClassifier_16S_trainsetNo18_QiimeFormat', 'RefOTUs.fa' ),
            join(or_dir, 'RDPClassifier_16S_trainsetNo18_QiimeFormat', 'Ref_taxonomy.txt' )
           ],
    "itgdb": [join(or_dir, "taxa_itgdb_seq.fasta"),
              join(or_dir, "taxa_itgdb_taxa.txt")
             ]
}

databases

{'silva': ['original_db/silva_138_NR99_seqs.fasta',
  'original_db/silva_138_NR99_taxa.txt'],
 'gg': ['original_db/gg_13_8_NR99_seqs.fasta',
  'original_db/gg_13_8_NR99_taxa.txt'],
 'gp_97': ['original_db/gp_97_otus_OLD.fasta',
  'original_db/gp_97_taxa_OLD.txt'],
 'rdp': ['original_db/RDPClassifier_16S_trainsetNo18_QiimeFormat/RefOTUs.fa',
  'original_db/RDPClassifier_16S_trainsetNo18_QiimeFormat/Ref_taxonomy.txt'],
 'itgdb': ['original_db/taxa_itgdb_seq.fasta',
  'original_db/taxa_itgdb_taxa.txt']}

### Inspect original files

In [4]:
for key, items in databases.items():
    taxafile = items[1]
    ! wc -l {taxafile}

436681 original_db/silva_138_NR99_taxa.txt
203452 original_db/gg_13_8_NR99_taxa.txt
127414 original_db/gp_97_taxa_OLD.txt
21195 original_db/RDPClassifier_16S_trainsetNo18_QiimeFormat/Ref_taxonomy.txt
110780 original_db/taxa_itgdb_taxa.txt


### Unify format of original dbs
Here we will save a formatted taxonomy file for Greengenes, RDP, the old gp97 database and the ITGDB-16S database from this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9580931/?report=classic). Those databases genus and species format is `g__Genus; s__specie`, while the format of the database we are creating and the Silva database is `g__Genus; s__Genus_specie`, so in this step we are only changing this format to make all the databases comparable. 

In greengenes and ITGDB database we will also include the taxonomy header required by qiime2: `Feature ID\tTaxon`.

In RDP, we are also converting the sequence to uppercase. 

In [5]:
dbs_format = {
    'silva': [join(or_dir, 'silva_138_NR99_seqs.fasta'),
             join(or_dir, 'silva_138_NR99_taxa.txt')
             ],
    'gg': [join(or_dir, "gg_13_8_NR99_seqs.fasta"),
           join(or_dir, "gg_13_8_NR99_taxa_spformat.txt")
          ],
    'gp_97': [join(or_dir, "gp_97_otus_OLD.fasta"),
           join(or_dir, "gp_97_taxa_OLD_spformat.txt")       
             ],
    'rdp': [ join(or_dir, 'rdp_16S_seqs.fasta' ),
            join(or_dir, 'rdp_16S_taxa_spformat.txt' )
           ],
    "itgdb": [join(or_dir, "taxa_itgdb_seq.fasta"),
              join(or_dir, "taxa_itgdb_taxa_spformat.txt")
             ]
}


In [6]:
%autoreload
for db, files in databases.items():
    if db == 'silva':
        continue
    
    # load database
    print(f'Formating {db}')
    database = utils.load_db_from_files(files[1], files[0])
    
    # format species level
    format_sp = ['gg', 'rdp', 'gp_97', 'itgdb']
    
    if db in format_sp:
        for ID, items in database.items():
            if items['taxa'][6] != "":
                items['taxa'][6] = '_'.join([items['taxa'][5], items['taxa'][6]])

    # convert sequence to uppercase
    if db == "rdp":
        for ID, items in database.items():
            items['seq'] = items['seq'].upper()
            
            
    # download formatted files
    utils.download_db_from_dict(database, dbs_format[db][1], dbs_format[db][0])
      

Formating gg
Formating gp_97
28362 entries without sequence or taxa information were deleted
Formating rdp
Formating itgdb


#### Create qza files

In [ ]:
dbs_format_qza = {}

for db, files in dbs_format.items():
    dbs_format_qza[db] = ["", ""]
    for i, file in enumerate(files):
        qza = utils.file_to_qza(file)
        dbs_format_qza[db][i] = qza

In [30]:
dbs_format_qza

{'silva': ['original_db/silva_138_NR99_seqs.qza',
  'original_db/silva_138_NR99_taxa.qza'],
 'gg': ['original_db/gg_13_8_NR99_seqs.qza',
  'original_db/gg_13_8_NR99_taxa_spformat.qza'],
 'gp_97': ['original_db/gp_97_otus_OLD.qza',
  'original_db/gp_97_taxa_OLD_spformat.qza'],
 'rdp': ['original_db/rdp_16S_seqs.qza',
  'original_db/rdp_16S_taxa_spformat.qza'],
 'itgdb': ['original_db/taxa_itgdb_seq.qza',
  'original_db/taxa_itgdb_taxa_spformat.qza']}

## Create merged database

We will create our database by merging RDP, Silva and Greengenes databases, in this order.

In [5]:
from copy import deepcopy
base_dbs = deepcopy(dbs_format)
base_dbs = {db: items for db, items in base_dbs.items() if db in ['rdp','silva','gg']}
base_dbs

{'silva': ['original_db/silva_138_NR99_seqs.fasta',
  'original_db/silva_138_NR99_taxa.txt'],
 'gg': ['original_db/gg_13_8_NR99_seqs.fasta',
  'original_db/gg_13_8_NR99_taxa_spformat.txt'],
 'rdp': ['original_db/rdp_16S_seqs.fasta',
  'original_db/rdp_16S_taxa_spformat.txt']}

### Filter base databases by taxonomy 
First we will filter the base databases to remove entries with taxonomies that we dont want to include in our database. 

#### Only Bacteria and Archae
Silva database contains Eukaryota and viruses

In [6]:
# Only modify taxa files

for db, files in base_dbs.items():
    taxa_file = files[1]
    taxa_out = f"{splitext(taxa_file)[0]}_BactArch.txt"
    ! grep "Bacteria\|Archaea" {taxa_file} > {taxa_out}
    
    # print
    ! wc -l {taxa_out}
    
    # rename base files
    base_dbs[db][1] = taxa_out



389016 original_db/silva_138_NR99_taxa_BactArch.txt
203452 original_db/gg_13_8_NR99_taxa_spformat_BactArch.txt
21194 original_db/rdp_16S_taxa_spformat_BactArch.txt


#### Filter weird names
Eliminate weird names (e.g. metagenome, uncultured,...)

In [7]:
patterns_file = "discard_patterns.txt"

# Only modify taxa files

for db, files in base_dbs.items():
    taxa_file = files[1]
    taxa_out = f"{splitext(taxa_file)[0]}_filtweird.txt"

    ! grep -vf {patterns_file} {taxa_file} > {taxa_out}
    
    # print
    ! wc -l {taxa_out}
    
    # rename base files
    base_dbs[db][1] = taxa_out

78415 original_db/silva_138_NR99_taxa_BactArch_filtweird.txt
201585 original_db/gg_13_8_NR99_taxa_spformat_BactArch_filtweird.txt
20944 original_db/rdp_16S_taxa_spformat_BactArch_filtweird.txt


#### Filter unknown species
Only database greengenes contains unknown species, but in any, we will pass the filter to both files. We suspect that unknown species from SILVA database were already remove with the previous step.

In [8]:
# Only modify taxa files
for db, files in base_dbs.items():
    taxa_file = files[1]
    taxa_out = f"{splitext(taxa_file)[0]}_filtsp.txt"

    ! grep -v "s__$" {taxa_file} > {taxa_out}
    
    # print
    ! wc -l {taxa_out}
    
    # rename base files
    base_dbs[db][1] = taxa_out

78415 original_db/silva_138_NR99_taxa_BactArch_filtweird_filtsp.txt
20443 original_db/gg_13_8_NR99_taxa_spformat_BactArch_filtweird_filtsp.txt
20151 original_db/rdp_16S_taxa_spformat_BactArch_filtweird_filtsp.txt


### Homogenize taxa nomenclature
#### Read db files

In [9]:
base_dbs_dict = {db: utils.load_db_from_files(paths[1], paths[0]) for db, paths in base_dbs.items()}

358265 entries without sequence or taxa information were deleted
183009 entries without sequence or taxa information were deleted
1044 entries without sequence or taxa information were deleted


#### Save process  
Now we are going to change the conda environment to run some especific python modules, therefore we will save our dictionary.

In [30]:
dbs_obj = join(or_dir, "dbs_dict.pickle")
pickle.dump(base_dbs_dict, open(dbs_obj, 'wb' ) )

#### CHANGE TO ETE3 ENVIRONMENT 
To switch between conda environments see [here](https://towardsdatascience.com/get-your-conda-environment-to-show-in-jupyter-notebooks-the-easy-way-17010b76e874). We will use the [ete3](http://etetoolkit.org/download/) python package
to obtain the lastest version of NCBI taxonomy.

In [1]:
import pickle
from os.path import join, basename
import update_taxonomy
from re import search
%load_ext autoreload

In [2]:
# Environment
or_dir = "original_db"

In [3]:
dbs_obj = join(or_dir, "dbs_dict.pickle")
dbs = pickle.load(open(dbs_obj, 'rb'))

#### Update taxonomy according NCBI (ete3 module)

In [6]:
%autoreload

not_found_taxa = []

for key, db in dbs.items():
    # Information
    found_taxa = 0
    change_taxa = 0
    len_original_db = len(db)
    print(f"Original size of {key}: {len_original_db}")
    
    # Copy database
    db_copy = db.copy()
    
    
    for ID, items in db.items():
        taxa = items['taxa']
        new_taxa = update_taxonomy.update_taxonomy_ncbi(taxa, join_taxa = False)
        
        
        if new_taxa:
            # check if taxa was changed
            if new_taxa != taxa:
                change_taxa +=1
                
#                 if not any('Candidatus' in tax for tax in new_taxa):
#                     print("old:", taxa)
#                     print("new:", new_taxa)
                    
                
        
                
            # check if there is still species level
            if new_taxa[6]:
                # replace taxa and count it
                db[ID]['taxa'] = new_taxa
                found_taxa = found_taxa + 1 
            else:
                db_copy.pop(ID)
                               
        else:
            not_found_taxa.append(taxa)
    
    dbs[key] = db_copy
    

    # Information
    len_db = len(db_copy)
    print(f"Final size of {key}: {len_db}")
    print(f"Total found taxa: {found_taxa}")
    print(f"{(change_taxa/len_db)*100} % of the taxa changed and was updated\n")


Original size of silva: 78415
Final size of silva: 76932
Total found taxa: 76893
64.62980294285863 % of the taxa changed and was updated

Original size of gg: 20443
Final size of gg: 20441
Total found taxa: 20441
57.365099554816304 % of the taxa changed and was updated

Original size of rdp: 20151
Final size of rdp: 20151
Total found taxa: 20151
49.54096570889782 % of the taxa changed and was updated



In [8]:
# Explore not found taxa
not_found_taxa

[['Bacteria',
  'Caldatribacteriota',
  'JS1',
  'JS1',
  'JS1',
  'JS1',
  'benzene_mineralizing'],
 ['Bacteria',
  'LCP-89',
  'LCP-89',
  'LCP-89',
  'LCP-89',
  'LCP-89',
  'saltmarsh_clone'],
 ['Bacteria',
  'Latescibacterota',
  'Latescibacterota',
  'Latescibacterota',
  'Latescibacterota',
  'Latescibacterota',
  'saltmarsh_clone'],
 ['Bacteria',
  'Cloacimonadota',
  'Cloacimonadia',
  'Cloacimonadales',
  'PBS-18',
  'PBS-18',
  'saltmarsh_clone'],
 ['Bacteria',
  'Cloacimonadota',
  'Cloacimonadia',
  'Cloacimonadales',
  'W27',
  'W27',
  'anaerobic_digester'],
 ['Bacteria',
  'Marinimicrobia_(SAR406_clade)',
  'Marinimicrobia_(SAR406_clade)',
  'Marinimicrobia_(SAR406_clade)',
  'Marinimicrobia_(SAR406_clade)',
  'Marinimicrobia_(SAR406_clade)',
  'hydrothermal_vent'],
 ['Bacteria',
  'Marinimicrobia_(SAR406_clade)',
  'Marinimicrobia_(SAR406_clade)',
  'Marinimicrobia_(SAR406_clade)',
  'Marinimicrobia_(SAR406_clade)',
  'Marinimicrobia_(SAR406_clade)',
  'hydrothermal_ve

#### Filter bacterium and archaeon species with only kingdom information  
There are some bacterias and archaeas that only contains information at kingdom and species levels, e.g. `k__Bacteria,..., s__bacterium_Te63R`. These entries are not informative. We will delete them.

In [9]:
for key, db in dbs.items():
    # Information
    deleted_items = 0
    len_original_db = len(db)
    print(f"Original size of {key}: {len_original_db}")
    
    # Copy database
    db_copy = db.copy()
    
    for ID, items in db.items():
        # if pattern is found and only kingdom information is available
        pattern_found = search('^bacterium|^archaeon', items['taxa'][6])
        notonly_kingdom_info = sum(bool(tax) for tax in items['taxa'][1:6])
        
        if pattern_found and not notonly_kingdom_info:
            # delete entry
            db_copy.pop(ID)
            
            # Information
            deleted_items = deleted_items + 1
    
    dbs[key] = db_copy
    
    # Information
    len_db = len(db_copy)
    print(f"Final size of {key}: {len_db}")
    print(f"Total deleted taxa: {deleted_items}")
    print(f"{(deleted_items/len_original_db)*100} % of the taxa was deleted\n")
            

Original size of silva: 76932
Final size of silva: 75061
Total deleted taxa: 1871
2.432017885925233 % of the taxa was deleted

Original size of gg: 20441
Final size of gg: 20441
Total deleted taxa: 0
0.0 % of the taxa was deleted

Original size of rdp: 20151
Final size of rdp: 20151
Total deleted taxa: 0
0.0 % of the taxa was deleted



#### Filter Kingdom  
Some eukaryota species where classified as bacteria (because the sequence belonged to a chloroplast o a mitochondria or a parasite of the eukaryota i guess) and with the ncbi taxonomy updated they have been reclassified as eukaryota. 
As we are not interested in eukaryota we will delete this entries. 

UPDATE: With further observations of the results, we have also noticed some Virus and kingdom unknown entries. Therefore we will only keep entries from kingdoms Bacteria and Archaea.

In [11]:
for key, db in dbs.items():
    # Information
    deleted_items = 0
    len_original_db = len(db)
    print(f"Original size of {key}: {len_original_db}")
    
    # Copy database
    db_copy = db.copy()
    
    for ID, items in db.items():
        # if pattern is found 
        
        if items['taxa'][0] not in ["Bacteria", "Archaea"]:
            # delete entry
            db_copy.pop(ID)            
            # Information
            deleted_items = deleted_items + 1
    
    dbs[key] = db_copy
    
    # Information
    len_db = len(db_copy)
    print(f"Final size of {key}: {len_db}")
    print(f"Total deleted taxa: {deleted_items}")
    print(f"{(deleted_items/len_original_db)*100} % of the taxa was deleted\n")
            

Original size of silva: 75061
Final size of silva: 74616
Total deleted taxa: 445
0.592851147733177 % of the taxa was deleted

Original size of gg: 20441
Final size of gg: 20434
Total deleted taxa: 7
0.03424489995597084 % of the taxa was deleted

Original size of rdp: 20151
Final size of rdp: 20151
Total deleted taxa: 0
0.0 % of the taxa was deleted



#### Save Updated taxonomy

In [13]:
dbs_obj = join(or_dir, "dbs_dict_updated.pickle")
pickle.dump(dbs, open(dbs_obj, 'wb' ) )

### Merge databases

#### Change environment 
 For further analysis, change to qiime2 environment.

In [1]:
import pickle
from os.path import join, basename, splitext, exists
from copy import deepcopy

from Bio import SeqIO
from Bio import Entrez

import new_approach
from qiime2 import Artifact
import pandas as pd
import numpy as np

import utils
%load_ext autoreload

#### Load databases object

In [2]:
or_dir = "original_db"
dbs_obj = join(or_dir, "dbs_dict_updated.pickle")
dbs = pickle.load(open(dbs_obj, 'rb'))

#### Merge RDP and Silva databases
We will use RDP as base database and we will add Silva entries with the following approach
 - For each entry in Silva look if the taxonomy is present in RDP.
     - NO: add entry to the database
     - YES: if there is already a taxonomy with this sequence
         - YES: skip
         - NO: add a new entry to the database

In [5]:
base_db = dbs['rdp']
candidate_db = dbs['silva']

In [7]:
# Merged db with initial SILVA entries
merged_db = deepcopy(base_db)

In [8]:
%autoreload
merged_db = new_approach.integrate_db(merged_db, candidate_db)

Initial len: 20151
Final len: 72721
Number of new taxonomies added: 3092
Number of new sequences added: 52570
Number of new substr found: 6523


#### Add Greengenes database

In [9]:
candidate_db = dbs['gg']
merged_db = new_approach.integrate_db(merged_db, candidate_db)

Initial len: 72721
Final len: 90269
Number of new taxonomies added: 288
Number of new sequences added: 17548
Number of new substr found: 1396


#### Download full 16S merged database

In [10]:
created_db = "created_db"
taxa_out = join(created_db, "gsr_full-16S_taxa.txt")
seqs_out = join(created_db, "gsr_full-16S_seqs.fasta")
utils.download_db_from_dict(merged_db, taxa_out, seqs_out)

## Add relevant vaginal species
### Obtain relevant species for vaginal
We obtain the relevant names form the supplementary taxa of this [article](https://doi.org/10.1186/1471-2164-13-S8-S17).

In [11]:
input_vaginal = "vaginal/vaginal_db.csv"
vaginal_df = pd.read_csv(input_vaginal, header = 2)
vaginal_df = vaginal_df[['GenBank Accession Number', 'Species-Level Taxon*']]

#### Preprocess species names: delete cluster and OTU terminologys

In [12]:
vaginal_df.replace(" cluster.*| OTU.*", "", inplace=True, regex=True)

#### Delete family names form nomenclature

In [13]:
vaginal_df.replace(".*aceae.*", "", regex=True, inplace=True)

#### Delete empty rows

In [14]:
vaginal_df.replace("", np.nan, inplace=True)
vaginal_df.replace("-", np.nan, inplace=True)
vaginal_df.dropna(inplace=True)

#### Keep only rows containning species

In [15]:
vaginal_df['length'] = vaginal_df['Species-Level Taxon*'].apply(lambda x: len(x.split(" ")))
vaginal_df = vaginal_df[vaginal_df['length'] > 1]
vaginal_df = vaginal_df[['GenBank Accession Number', 'Species-Level Taxon*']]

#### Save list of vaginal species

In [16]:
vagina_out = "vaginal/vaginal_filtered_species.csv"
if not exists(vagina_out):
    vaginal_df.to_csv(vagina_out, index=False, header=True)
else:
    vaginal_df = pd.read_csv(vagina_out, header=0)

### Obtain taxa from NCBI 
First, we will check in the NCBI if the vaginal_id has some NCBI taxid associated. 

In [17]:
vaginal_sp = list(vaginal_df['Species-Level Taxon*'])
vaginal_ids = list(vaginal_df['GenBank Accession Number'])

In [18]:
Entrez.email = "leidyalejandra.gonzalez01@estudiant.upf.edu"

SEVERAL NOTES ON THE CHUNK BELOW
1. ncbi api is a bitch
2. most of the time it will work but sometimes it will start an infinite loop. SO: check the output and if you see the loop is happening stop and run again

In [ ]:
tax_ids = []

# FIND NCBI TAX IDS

for i in range(len(vaginal_ids)):
    
    # 1. search the taxonomy id for the genbank id
    # we are sure that genbank ids exist in ncbi database, so the result cannot be empty 
    # if the result is empty (aka record[0] raises a Index error) try again
    # it no error is raised, exit the loop

    while True:
        try:
            record = Entrez.read(Entrez.elink(db="taxonomy", dbfrom="nucleotide", id=vaginal_ids[i]))
            record[0]
        except IndexError:
            print(f"{vaginal_ids[i]} entry not found, trying again")
            continue
        else:
            print(f"{vaginal_ids[i]} entry found")
            break 
    
    # 2. Look for the taxaid in the result
    # if the result contains the taxID, save it
    # if not, 2 things may have happened:
        # 1: the sequence has been updated and we need to search the updated genbank ID
        # 2: an error occurred *angry noises* (YES IT HAPPENS SOMETIMES)
    # To be sure that it is not an error, we will repeat the search a maximum of 3 times
    # (there is still a possibility that the error occurs but it is small)
    
    
    # If record found
    if record[0]['LinkSetDb']:
        tax_ids.append(record[0]['LinkSetDb'][0]['Link'][0]['Id'])
        
    # Not found: 
    else:
        
        # try 3 more times to check it is not an error:
        for x in range(3):
            print(f"{vaginal_ids[i]} taxonomy not found, trying again")
            record = Entrez.read(Entrez.elink(db="taxonomy", dbfrom="nucleotide", id=vaginal_ids[i]))
            # if found end loop
            if record[0]['LinkSetDb']:
                print(f"{vaginal_ids[i]} taxonomy found")
                break
        
        
        # If now taxonomy is found: save
        if record[0]['LinkSetDb']:
            tax_ids.append(record[0]['LinkSetDb'][0]['Link'][0]['Id'])
            
        # If taxonomy still not found: updated sequence
        else:
            print(f"{vaginal_ids[i]} taxonomy not found, trying updated version")
            # Change extension to updated version (1 to 2)
            vaginal_ids[i] = f"{vaginal_ids[i][:-1]}2"
            
            # search with the updated id until it is found
            
            while True:
                try:
                    record = Entrez.read(Entrez.elink(db="taxonomy", dbfrom="nucleotide", id=vaginal_ids[i]))
                    record[0]['LinkSetDb']
                except IndexError:
                    print(f"{vaginal_ids[i]} entry not found, trying again")
                    continue
                else:
                    print(f"{vaginal_ids[i]} entry found")
                    break
                
            # if it has taxonomy,save, if not fuck it

            if record[0]['LinkSetDb']:
                tax_ids.append(record[0]['LinkSetDb'][0]['Link'][0]['Id'])
            else:
                print(f"{vaginal_ids[i]} taxonomy not found")

Looking manually in the NCBI Nucleotide database, we suspected that the majority of not found records correspond to updated sequences. As an example, `M59083.1` sequence was updated to `M59083.2`, changing its extension id from `.1` to `.2`. Therefore we will change the extension of the not found records in order to search them again.

In [20]:
len(tax_ids) == len(vaginal_ids)

True

Downloand objects. Note: for some unknown and strange reason, we weren't able to pickle de tax_ids, so, we decided to save the variable in a script.

In [21]:
vaginal_id_obj = join("vaginal", "vaginal_ids.pickle")
pickle.dump(vaginal_ids, open(vaginal_id_obj, 'wb' ) )

In [22]:
# wtf
with open("vaginal/vaginal_sp_taxid.py", "w") as file:
    file.write(f"tax_ids = {tax_ids}")

### Create vaginal database
**Change to ete3 environment**

In [1]:
import pickle
from os.path import join, basename
from re import search

from Bio import Entrez
from Bio import SeqIO

import update_taxonomy
import utils
%load_ext autoreload

#### Load sequence ids and taxonomy ids

In [2]:
vaginal_id_obj = join("vaginal", "vaginal_ids.pickle")
vaginal_ids = pickle.load(open(vaginal_id_obj, 'rb'))

In [3]:
from vaginal.vaginal_sp_taxid import tax_ids
vaginal_txid = tax_ids

#### Obtain sequences from the ncbi

In [4]:
Entrez.email = "leidyalejandra.gonzalez01@estudiant.upf.edu"
# Get Seqs from NCBI
fasta_fh = Entrez.efetch(db="nucleotide", id=vaginal_ids, rettype="fasta", retmode="text")

ncbi_seqs = SeqIO.to_dict(SeqIO.parse(fasta_fh, "fasta"))

#### Create vaginal database

In [5]:
vaginal_seqs = "vaginal/vaginal_seqs.fasta"
vaginal_tax = "vaginal/vaginal_taxa.txt"
vaginal_db = {}

In [6]:
%autoreload
del_seqs = 0
print(f"Initial sequences: {len(vaginal_ids)}")

for ID, tax_id in zip(vaginal_ids, vaginal_txid):
    seq = ncbi_seqs[ID].seq
    taxa = update_taxonomy.get_taxa_from_specie_taxid(tax_id)
    
    # discard complete genomes and unknown species
    if len(seq) < 2000 and taxa[6] != 'uncultured_bacterium' : 
        # add entry
        vaginal_db[ID] = {'seq': seq, 'taxa': taxa}     
       
    else:
        del_seqs = del_seqs + 1

print(f"Discarded sequences (complete genomes or unknown species): {del_seqs}")  

Initial sequences: 634
Discarded sequences (complete genomes or unknown species): 15


In [ ]:
vaginal_db

#### Downloand vaginal database

In [9]:
utils.download_db_from_dict(vaginal_db, vaginal_tax, vaginal_seqs)

### Integrate vaginal db

In [1]:
# Change to qiime2 env 
import pickle
from os.path import join, basename, splitext, exists

from Bio import SeqIO
from Bio import Entrez

from libs import new_approach
from qiime2 import Artifact
import pandas as pd
import numpy as np

import utils

In [2]:
# prevously merged database (full 16S database)
taxa_16S = "created_db/gsr_full-16S_taxa.txt"
seqs_16S = "created_db/gsr_full-16S_seqs.fasta"
# vaginal database
vaginal_seqs = "vaginal/vaginal_seqs.fasta"
vaginal_tax = "vaginal/vaginal_taxa.txt"

merged_db = utils.load_db_from_files(taxa_16S, seqs_16S)
vaginal_db = utils.load_db_from_files(vaginal_tax, vaginal_seqs)

# set(merged_db.keys()).intersection(set(vaginal_db.keys()))

merged_db = new_approach.integrate_db(merged_db, vaginal_db)

Initial len: 90269
Final len: 90408
Number of new taxonomies added: 8
Number of new sequences added: 139
Number of new substr found: 57


#### Download vaginal integrated db

In [3]:
taxa_out = "created_db/gsrv_full-16S_taxa.txt"
seqs_out = "created_db/gsrv_full-16S_seqs.fasta"

utils.download_db_from_dict(merged_db, taxa_out, seqs_out)

# convert files to qza
utils.file_to_qza(taxa_out)
utils.file_to_qza(seqs_out)

'created_db/gsrv_full-16S_seqs.qza'

# Filter full-16S database by length 

As we have seen in the [database stats](gsrv_stats.ipynb) notebook, full-16S database has some entries with lengths that are not probable to correspond to 16S rRNA sequence. For this reason, we will filter out too short and too long sequences

In [1]:
from utils import check_dir
import utils
from os.path import join, exists, basename, splitext
import pickle
from re import sub, search
from copy import copy
%load_ext autoreload

In [2]:
# load database
taxa_path = "created_db/gsrv_full-16S_taxa.txt"
seqs_path = "created_db/gsrv_full-16S_seqs.fasta"
db = utils.load_db_from_files(taxa_path=taxa_path, seqs_path=seqs_path)

In [3]:
# filter database by sequence length
minlen = 900
maxlen = 1800

db_filt = db.copy()

for ID, items in db.items():
    seqlen = len(items['seq'])
    if seqlen < minlen or seqlen > maxlen:
        db_filt.pop(ID, None)

In [4]:
print(f"Previous gsr size: {len(db)}")
print(f"Filtered gsr size: {len(db_filt)}")

Previous gsr size: 90408
Filtered gsr size: 90231


In [5]:
taxa_out = "created_db/gsrv_full-16S_filt_taxa.txt"
seqs_out = "created_db/gsrv_full-16S_filt_seqs.fasta"

utils.download_db_from_dict(db_filt, taxa_out, seqs_out)

In [6]:
# convert files to qza
utils.file_to_qza(taxa_out)
utils.file_to_qza(seqs_out)

'created_db/gsrv_full-16S_filt_seqs.qza'

# Accomodate Metasquare taxonomy for qiime training and classification

Many Metasquare entries have incomplete taxonomical anotations (aka they are missing taxonomical levels) and this becomes problematic when performing classification with qiime2. Therefore here we complete all metasquare taxonomies up to species level, filling the missing levels with "unknown".

In [40]:
import utils

In [41]:
# load taxa file
ms_taxafile = 'created_db/metasquare_V4_taxa.txt'

ms_taxa = utils.load_db_from_files(taxa_path=ms_taxafile)

In [42]:
# fill taxonomies
for key in ms_taxa.keys():
    a = ms_taxa[key]['taxa'] 
    ms_taxa[key]['taxa'] = [a[i] if i < len(a) else 'unknown' for i in range(0,7)]

In [43]:
# download formated taxa
ms_taxa_out = 'created_db/metasquare_V4_taxa_format.txt'
utils.download_db_from_dict(d = ms_taxa, taxa_out_file=ms_taxa_out)

In [44]:
# import to qza
utils.file_to_qza(ms_taxa_out)

'created_db/metasquare_V4_taxa_format.qza'

# Create taxonomy file for GTDB + format sequence file

In [35]:
import utils
from Bio import SeqIO

In [36]:
in_file = 'original_db/gtdb_r207_seqs.fasta'

In [37]:
# create database
gtdb = {}

with open(in_file, "r") as seqsfile:
    for record in SeqIO.parse(seqsfile, "fasta"):
        ID = record.id
        seq= record.seq
        desc = record.description
        taxa = "_".join(desc.split(' ')[1:-3])
        taxa = taxa.split(';')
        taxa = utils.rm_prefs(taxa)
        
        gtdb[ID] = {}
        gtdb[ID]['taxa'] = taxa
        gtdb[ID]['seq'] = seq
        

In [38]:
# download database
gtdb_taxa_out = 'original_db/gtdb_full-16S_taxa.txt'
gtdb_seq_out = 'original_db/gtdb_full-16S_seqs.fasta'

utils.download_db_from_dict(d=gtdb, taxa_out_file=gtdb_taxa_out, seqs_out_file=gtdb_seq_out)

In [39]:
# import to qza
utils.file_to_qza(gtdb_taxa_out)
utils.file_to_qza(gtdb_seq_out)

'original_db/gtdb_full-16S_seqs.qza'